# 코랩에서 TPU사용하기
- Colab에서 런타임 > 런타임 유형 변경 > 하드웨어 가속기에서 'TPU' 선택

In [ ]:
# TPU초기화
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

#  TPU Strategy 셋팅
strategy = tf.distribute.TPUStrategy(resolver)

### 딥 러닝 모델을 컴파일을 할 때도 추가적인 코드가 필요
- 모델의 컴파일은 strategy.scope 내에서 이루어져야 함

In [ ]:
#모델 쌓기
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

# 컴파일
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

#이후 fit()하면 TPU버전으로

## 안끊기게 하기
- F12 -> 콘솔탭에 밑에꺼 붙여넣기 -> F12창 계속 켜놯야함  
function ClickConnect(){  
console.log("Working");   
document.querySelector("colab-toolbar-button#connect").click()   
}setInterval(ClickConnect, 1800000)  


# BERT 기본 사용법

# 1. 토크나이징하기
- BertTokenizer, RobertaTokenizer, GPT2Tokenizer, AlbertTokenizer, AutoTokenizer등

In [ ]:
# 임포트
from transformers import BertTokenizer # bert용 wordpiece기반 : [CLS]-단어-[SEP]
from transformers import BertTokenizerFast # 빠른버전
from transformers import AutoTokenizer # 추천!!! 거의 모든 종류에 사용가능(자동으로 추천해줌)

# 사전학습정보 불러오기
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # 이 안에 이름

# 토크나이징 하기
encoded = tokenizer(text,                 #하나의 문자열 or 문자들의 list
                    padding='max_length', # True 또는 'longest': 가장 긴 길이로, max_length: 안에 따로 max_length=n파라미터 필요, False: 안함
                    truncation=True, # 지정한 padding값이나 512길이가 넘어가면 자름(앞에서부터)
                    max_length=128, # 패딩 길이
                    return_tensors='tf') # tf:텐서플로우 들어갈때, pt: pytorch들어갈때, np: 넘파이 형태로 반환

# 디코딩하기(숫자를 다시 원래 단어로)
decoded = tokenizer.decode(encoded['input_ids'][n].numpy().tolist(), # n-1번째 샘플 디코딩 -> tf형태이므로 넘파이로 바꾸고 다시 리스트로
                           skip_special_tokens=True) # 특수토큰 제외

### encoded값
{  
  'input_ids': [...], # 값이 숫자인덱스로 변환된 데이터  
  'attention_mask': [...], # 의미있는 토큰=1, 패딩된 토큰은 0  
  'token_type_ids': [...]  # 각 문장을 0부터 1씩 증가하게 표현, 문장쌍일 때만 의미 있음  
}  

### 예시
{  
  'input_ids': [  
    [101, 1045, 2293, 102, 0, 0],        # 첫 번째 샘플의 토큰 ID 시퀀스  
    [101, 2026, 2171, 2003, 102, 0],    # 두 번째 샘플  
    [101, 2009, 2001, 2204, 102, 0]     # 세 번째 샘플  
  ],  
  'attention_mask': [  
    [1, 1, 1, 1, 0, 0],  # 첫 번째 샘플: 마지막 두 개는 패딩이라 0  
    [1, 1, 1, 1, 1, 0],  # 두 번째 샘플  
    [1, 1, 1, 1, 1, 0]   # 세 번째 샘플  
  ],  
  'token_type_ids': [  
    [0, 0, 0, 0, 0, 0],  # 문장 단일 입력이라 모두 0  
    [0, 0, 0, 0, 0, 0],  
    [0, 0, 0, 0, 0, 0]  
  ]  
}  

# 2. BERT여러 모델들
- 파이토치용은 그냥 Bert~
- 텐서플로우용은 TFBert~

### 2-1. BERT기본 모델: TFBertModel
- tokenizer의 dict형태 그대로 넣거나 !!!!!!
- input_ids와 attention_mask 두개만 넣어도 됌!!!!!!!!!!!! (token_type_ids 디폴트는 0)

In [ ]:
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello, BERT!", return_tensors="tf")

# 토크나이징 된걸 그냥 model에 넣으면 됨
outputs = model(inputs)

# 기본 BERT의 마지막 층 출력
last_hidden_states = outputs.last_hidden_state   #모든 토큰(cls,sep등 포함)에 대한 최종 히든 벡터: (batch_size, seq_len, hidden_size)
pooler_output = outputs.pooler_output            #샘플별 맨앞 하나 존재하는 cls에 추가적인 층과 tanh활성화를 거친 벡터
                                                 # -> 그 샘플의 문장을 대표하는 벡터: (batch_size, hidden_size)

### 2-2. TFBertForSequenceClassification(문장 분류용 층이 추가)
- 입력(다) 대 출력(일)
- 출력값은 클래스별 logits(z값)
### cls!!!값을 추가 dense+여러 구성에 넣어 분류한것

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=n) # 이진분류면 1 or 2, 다중분류면 해당 클래스 개수 n

inputs = tokenizer("Hello, BERT!", return_tensors="tf")
outputs = model(inputs)

#최종 출력층 값
logits = outputs.logits   # (batch_size, num_labels)==(샘플수, n)

# logits → 확률 변환 (softmax)
probabilities = tf.nn.softmax(logits, axis=-1) # 마지막차원에 적용하는 축-> 각 클래스별 확률구함

# 각 샘플별 예측 클래스 인덱스
predicted_class = tf.argmax(probabilities, axis=-1).numpy()[0]

#### num_labels=1일경우 시그모이드(이진분류는 num_labels=2하고 그냥 소프트맥스해도됨)
# sigmoid로 확률 변환
probability = tf.sigmoid(logits)[0][0].numpy()

# 예측 클래스 (threshold 0.5)
predicted_class = 1 if probability > 0.5 else 0

## 이때 최종값으로 logits값이 나오지만 fit()할때는 compile에 loss값을 넣어주므로 SparseCategoricalCrossentropy, BinaryCrossentropy등에 의해 자동으로 확률-> 클래스로 변환해 정답값과 비교할수있게됨

### 2-3. TFBertForTokenClassification(토큰분류용)
- 각 토큰마다의 레이블 예측(다 대 다)
- 품사 태깅 등

In [ ]:
from transformers import BertTokenizer, TFBertForTokenClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=n) # 각 토큰이 분류될수있는 클래스 수!!!!

inputs = tokenizer("Hello, BERT!", return_tensors="tf")
outputs = model(inputs)

# logits[i][j]는 i번째 문장의 j번째 토큰
logits = outputs.logits   # (batch_size, seq_len, num_labels) -> 문장의 각 토큰별 n개의 분류점수값

# 각 토큰별 예측 클래스 보기
predicted_classes = tf.argmax(logits, axis=-1)  #(batch_size, seq_len) 예시) 하나의 문장에서 [1,0,0,2,4,0,3]

# 확률로 보기
probs = tf.nn.softmax(logits, axis=-1)  # (batch, seq_len, num_labels)

### 2-4. TFBertForQuestionAnswering(질문 답변용 모델)
- 입력 문장 내에서 정답의 시작/끝 위치 예측

In [ ]:
from transformers import BertTokenizer, TFBertForQuestionAnswering

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForQuestionAnswering.from_pretrained("bert-base-uncased")

question = "Who was Jim Henson?"
text = "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="tf") # 질문, 정답이 들은 문맥으로 줌 -> [CLS] 질문 [SEP] 문맥 [SEP]형태
outputs = model(inputs)

start_logits = outputs.start_logits  # (batch_size, seq_len) :각 토큰이 "답 시작"일 logit 점수
end_logits = outputs.end_logits      # (batch_size, seq_len) : 각 토큰이 "답 끝"일 logit 점수

# 가장 높은 점수의 토큰 찾기
start_index = tf.argmax(outputs.start_logits, axis=-1).numpy()[0]
end_index = tf.argmax(outputs.end_logits, axis=-1).numpy()[0]

# 토큰-> 원문단어로
input_ids = inputs["input_ids"][0]  # 1번째 샘플 토큰 ID 시퀀스
tokens = tokenizer.convert_ids_to_tokens(input_ids) # 숫자 리스트-> 문자열 리스트

answer_tokens = tokens[start_index : end_index + 1]
answer = tokenizer.convert_tokens_to_string(answer_tokens) # 문자열 리스트 -> 문자열로

print("Predicted Answer:", answer)

### 2-5. 커스텀 출력층 추가 (TensorFlow)

In [ ]:
import tensorflow as tf
from transformers import TFBertModel

class TFBertClassifier(tf.keras.Model):
    def __init__(self, num_labels=2):
        super().__init__()
        self.bert = TFBertModel.from_pretrained("bert-base-uncased")  # 기본 모델
        self.classifier = tf.keras.layers.Dense(num_labels)   # 추가해줄 층, 활성화 함수는 없음

    def call(self, inputs):
        outputs = self.bert(inputs)
        pooled_output = outputs.pooler_output  # (batch_size, hidden_size), 처리된 cls벡터
        logits = self.classifier(pooled_output)
        return logits

# 사용 예시
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertClassifier(num_labels=2)

inputs = tokenizer("Hello TensorFlow BERT!", return_tensors="tf")
logits = model(inputs)

# 3. 컴파일하기
- 마지막 층이 활성화 함수 안거친 logits이면(activation=None) loss에 from_logits=True(자동으로 활성화함수 적용시켜줌) !!!!!!
- 마지막 층이 활성화 함수 거쳤으면 loss에 from_logits=False !!!!!!

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # or BinaryCrossentropy
    metrics=["accuracy"]  #auc등 존재
)

# 콜백
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # 검증 손실 모니터링, val_auc도 가능
    patience=3,          # 3번 연속 개선 없으면 종료
    restore_best_weights=True)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.h5',  # 저장할 파일 이름
    monitor='val_loss',
    save_best_only=True)       # 가장 좋은 모델만 저장


# 4. tf.data.Dataset으로 fit에 넣을 형태로 만들기
- (입력, 라벨)쌍의 튜플 구조
- 입력은 여러개도 가능(input_ids, attention_mask, token_type_ids)

In [ ]:
# train, val, test 분리하기(80%,20%)
from sklearn.model_selection import train_test_split # 이걸로 분리

#이후 train,val,test에 각각 밑의 코드 실행
# Dataset만들기
encoded = tokenizer(texts) # train, val, test별로 다 별도 진행
labels = tf.convert_to_tensor(labels) # 숫자정답컬럼을 텐서로

# dict로 { 'input_ids': ..., 'attention_mask': ..., 'token_type_ids': ... }구조로
dataset = tf.data.Dataset.from_tensor_slices((dict(encoded), labels)) # 각 element는 (inputs_dict, label) 튜플 형태

# 배치 & 셔플 & 반복 등 처리
dataset = dataset.shuffle(100).batch(32).prefetch(tf.data.AUTOTUNE) # dataset은 fit대신 여기서 배치사이즈 지정 !!!!

# 5. 학습과 저장 불러오기

In [ ]:
# 학습
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3,
    callbacks=[early_stop, model_checkpoint] 
)

# 저장(hugging face방식): ./saved_model/ 폴더에 모델 가중치(tf_model.h5), config, vocab 등이 저장
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

# 불러오기
from transformers import TFBertForSequenceClassification, BertTokenizer

model = TFBertForSequenceClassification.from_pretrained("./saved_model")
tokenizer = BertTokenizer.from_pretrained("./saved_model")


### 커스텀 모델일경우 !!!
model.save("custom_bert.h5") # .h5로 저장

from your_code import TFBertClassifier  # 정의해놓은 클래스

model = TFBertClassifier(num_labels=2)
model.load_weights("custom_bert.h5")

# 6. 평가와 예측

In [ ]:
# 평가
loss, acc = model.evaluate(val_dataset) #손실과 정확도

# 예측: 값이 test_dataset로 여러개일땐 model.predict/ 하나일땐 직접 호출
def predict_bert(model, tokenizer, texts):
    enc = tokenizer(texts, return_tensors="tf", padding=True, truncation=True)
    logits = model(enc).logits
    preds = tf.argmax(tf.nn.softmax(logits, axis=-1), axis=-1)
    return preds.numpy()

predict_bert(model,tokenizer,texts)

# 7. 중요 텍스트 시각화
- 입력 데이터의 어느부분이 중요한지 보여줌

## bertviz로 Attention 시각화

In [ ]:
!pip install bertviz

In [ ]:
from transformers import BertTokenizer, TFBertModel
from bertviz import head_view

# 1) 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# 2) 입력 문장 토크나이징
sentence = "The quick brown fox jumps over the lazy dog"
inputs = tokenizer(sentence, return_tensors='tf')

# 3) 모델에 입력하고 attention weight 얻기
outputs = model(inputs, output_attentions=True)
attentions = outputs.attentions  # tuple: (layers, batch, heads, seq_len, seq_len)

# 4) 토큰 리스트 추출
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].numpy())

# 5) bertviz로 시각화
head_view(attentions, tokens)

## tf.GradientTape()으로 기울기 이용

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# 모델과 토크나이저 로드
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "This is an example sentence."
inputs = tokenizer(text, return_tensors="tf")

#임베딩 레이어 참고
embedding_layer = model.bert.embeddings

with tf.GradientTape() as tape:
    inputs_embeds = embedding_layer(inputs['input_ids'])
    tape.watch(inputs_embeds)
    outputs = model(inputs_embeds=inputs_embeds)
    loss = outputs.logits[:, 1]

grads = tape.gradient(loss, inputs_embeds)

## SHAP (SHapley Additive exPlanations)로 각 토큰 별 기여도를 측정

In [ ]:
!pip install shap

In [ ]:
import shap
import tensorflow as tf

# tokenizer, model 준비 완료 상태 가정
inputs = tokenizer(..., return_tensors="tf")

def f(input_ids_np):
    # numpy 배열 → tf.Tensor 변환
    inputs_tf = {"input_ids": tf.convert_to_tensor(input_ids_np)}
    outputs = model(inputs_tf)
    probs = tf.nn.softmax(outputs.logits, axis=-1)
    return probs.numpy()

# 배경 데이터: input_ids만 추출 (배치 크기 10)
background = inputs["input_ids"][:10].numpy()

explainer = shap.KernelExplainer(f, background)
shap_values = explainer.shap_values(inputs["input_ids"].numpy())

# 시각화 (샘플 0 기준)
shap.summary_plot(shap_values[0], inputs["input_ids"].numpy())